In [ ]:
import urllib as ur
import json
import pandas as pd
import datetime
import pytz

In [ ]:
# Converte hora dada a time zone atual, a zona a ser convertida e a diferença de tempo
def convert_datetime_timezone(dt, tz1, tz2):
    """
    Converte uma hora no fuso UTC ou São Paulo para um provável fuso de Vancouver.
    
    Parameters
    ------------
    dt : unix timestamp
        Timestamp a ser convertido para outro fuso horário.
    
    tz1, tz2 : Timezone String
        Time zone atual e a que a hora irá ser convertida.
    
    Returns
    ----------
    dt : unix timestamp
        Timestamp já convertida para o fuso de Vancouver.
    
    """    
    
    tz1 = pytz.timezone(tz1)
    tz2 = pytz.timezone(tz2)

    dt = datetime.datetime.fromtimestamp(dt)
    dt = datetime.datetime.strptime(str(dt),"%Y-%m-%d %H:%M:%S")
    dt = tz1.localize(dt)
    dt = dt.astimezone(tz2)
    
    try:
        # Fuso horário comum de Vancouver
        dt = datetime.datetime.strptime(str(dt),"%Y-%m-%d %H:%M:%S-08:00")
    except:
        # Fuso horário característico de horário de verão em Vancouver
        dt = datetime.datetime.strptime(str(dt),"%Y-%m-%d %H:%M:%S-07:00")
    
    dt = int(dt.timestamp())

    return dt

In [ ]:
# Coletando a hora atual de vancouver
vanc_tz = pytz.timezone('America/Vancouver')
time = datetime.datetime.now(tz=vanc_tz)

In [ ]:
# Requisição a api
try:
    i = 0
    response = None
    # Loop para evitar requisições demoradas
    while (response == None and i < 5):
        response = ur.request.urlopen('https://bookit.modo.coop/api/v2/availability', timeout=20).read().decode('UTF-8')
        json_Availability = json.loads(response)
        
        # Adquirindo todos os ids dos carros 
        car_ids = list(json_Availability['Response']['Availability'].keys())

        i += 1
except Exception as e:
    print('Erro na requisição: ' + str(e))

In [ ]:
cars = []
start = []
end = []
location = []

i = 0
# Percorrendo todos os ids dos carros para adquirir seus intervalos de disponibilidade
while (i < len(car_ids)):
    carAvailability = json_Availability['Response']['Availability'][str(car_ids[i])]['Availability']
    
    # Loop para caso haja mais de um intervalo de disponibilidade
    for j in range(len(carAvailability)):
        cars.append(car_ids[i])
        start.append(convert_datetime_timezone(int(carAvailability[j]['StartTime']), 'America/Sao_Paulo', 'America/Vancouver'))
        end.append(convert_datetime_timezone(int(carAvailability[j]['EndTime']), 'America/Sao_Paulo', 'America/Vancouver'))
        location.append(carAvailability[j]['LocationID'])
    
    i += 1
    
cars = pd.DataFrame(cars, columns=['CarID'])
start = pd.DataFrame(start, columns=['StartTime'])
end = pd.DataFrame(end, columns=['EndTime'])
location = pd.DataFrame(location, columns=['LocationID'])

In [ ]:
# Valores de inicio e fim da requisição
req_start = convert_datetime_timezone(int(json_Availability['Request']['start']), 'America/Sao_Paulo', 'America/Vancouver')
req_end = convert_datetime_timezone(int(json_Availability['Request']['end']), 'America/Sao_Paulo', 'America/Vancouver')

In [ ]:
# Armazenando a hora da captura e seu timestamp
cap_time = []
cap_timestamp = []
request_start = []
request_end = []

for i in range(len(cars)):
    cap_time.append(time)
    request_start.append(req_start)
    request_end.append(req_end)
    
cap_time = pd.DataFrame(cap_time, columns=['Capture_time'])
request_start = pd.DataFrame(request_start, columns=['RequestStart'])
request_end = pd.DataFrame(request_end, columns=['RequestEnd'])

In [ ]:
data = pd.concat([cap_time, location, cars, start, end, request_start, request_end], axis=1)

In [ ]:
from os.path import exists 

# Verificando se já existe o arquivo, para evitar a repetição dos labels das colunas
if (exists('modo_data.csv')):
    data.to_csv('modo_data.csv', mode='a', header=False, index=False, encoding='utf-8')
else:
    data.to_csv('modo_data.csv', index=False, encoding='utf-8')